In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np

In [2]:
from nuplan_extent.planning.training.modeling.models.decoders.gameformer_decoder import GameFormerDecoder0, GameFormerDecoderK

outputs from encoder:

agent_context_encoding: [11, B, 256]

scene_context_encoding: [91, B, 256]

scene_context_mask: [B, 91]

In [17]:
a = nn.MultiheadAttention(256, num_heads=4)

In [21]:
list(a.parameters())

[Parameter containing:
 tensor([[-0.0151,  0.0619,  0.0157,  ..., -0.0198, -0.0080, -0.0393],
         [-0.0606, -0.0054, -0.0268,  ..., -0.0210,  0.0118, -0.0351],
         [-0.0595,  0.0741,  0.0233,  ...,  0.0400, -0.0144,  0.0231],
         ...,
         [ 0.0419,  0.0182, -0.0268,  ...,  0.0481, -0.0336,  0.0575],
         [ 0.0387,  0.0700, -0.0591,  ...,  0.0160,  0.0463, -0.0269],
         [-0.0050, -0.0416,  0.0266,  ...,  0.0126,  0.0584,  0.0525]],
        requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [3]:
B = 5
num_context = 91
num_agents = 11
D = 256

In [4]:
context_encoding = {
    "agent_context_encoding": torch.randn(num_agents, B, D),
    "scene_context_encoding": torch.randn(num_context, B, D),
    "scene_context_mask": torch.randn(B, num_context)
}

In [8]:
decoder0 = GameFormerDecoder0(num_agents, 50)

output = decoder0(context_encoding)

print(output['scores_0'].shape)
print(output['trajectories_0'].shape)
print(output['query_content_features_0'].shape)

torch.Size([5, 11, 6, 1])
torch.Size([5, 11, 6, 50, 6])
torch.Size([5, 11, 6, 256])


Decoder-K
---

In [9]:
fake_shared_mlp =nn.Sequential(
    nn.Linear(50 * 2, D),
    nn.ReLU6(),
    nn.Linear(D, D),
    nn.ReLU6(),
    nn.MaxPool2d(kernel_size=[1, 3], stride=1, padding=[0, 1]),
)
fake_context_encoding = {
    "agent_context_encoding": torch.randn(num_agents, B, D),
    "scene_context_encoding": torch.randn(num_context, B, D),
    "scene_context_mask": torch.randn(B, num_context) > 0.5,
}

In [11]:
multi_future_encoding = fake_shared_mlp(output['trajectories_0'][..., :2].reshape(5, 11, 6, 100))
fake_context_encoding["multi_future_encoding"] = multi_future_encoding

In [12]:
for key, val in fake_context_encoding.items():
    print(key, val.shape)
print("=====")
for key, val in output.items():
    print(key, val.shape)

agent_context_encoding torch.Size([11, 5, 256])
scene_context_encoding torch.Size([91, 5, 256])
scene_context_mask torch.Size([5, 91])
multi_future_encoding torch.Size([5, 11, 6, 256])
=====
scores_0 torch.Size([5, 11, 6, 1])
trajectories_0 torch.Size([5, 11, 6, 50, 6])
query_content_features_0 torch.Size([5, 11, 6, 256])


In [14]:
decoderk = GameFormerDecoderK(50, 256, k=1)

decoderk_output = decoderk(fake_context_encoding, output)

In [15]:
for key, val in decoderk_output.items():
    print(key, val.shape)

scores_1 torch.Size([5, 11, 6, 1])
query_content_features_1 torch.Size([5, 11, 6, 256])
trajectories_1 torch.Size([5, 11, 6, 50, 6])
